In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/e-commerce-shoppers-behaviour-understanding/test_data_v2.csv
/kaggle/input/e-commerce-shoppers-behaviour-understanding/train_data_v2.csv
/kaggle/input/e-commerce-shoppers-behaviour-understanding/sample.csv


In [2]:
import pandas as pd
import numpy as np

from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer,KNNImputer
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.preprocessing import OrdinalEncoder



# **LOADING DATASETS**

In [3]:
train_data=pd.read_csv("/kaggle/input/e-commerce-shoppers-behaviour-understanding/train_data_v2.csv")
test_data=pd.read_csv("/kaggle/input/e-commerce-shoppers-behaviour-understanding/test_data_v2.csv")
y_train=train_data[train_data.columns[-1]]
x_train=train_data.drop(['Made_Purchase'],axis=1)
x_test=test_data[test_data.columns]

print(x_train.shape[1]==x_test.shape[1])
print(x_train.shape)
print(type(x_train))

True
(14731, 21)
<class 'pandas.core.frame.DataFrame'>


# **ANALYZING THE DATA**

In [4]:
x_train.dtypes

HomePage                           float64
HomePage_Duration                  float64
LandingPage                        float64
LandingPage_Duration               float64
ProductDescriptionPage             float64
ProductDescriptionPage_Duration    float64
GoogleMetric:Bounce Rates          float64
GoogleMetric:Exit Rates            float64
GoogleMetric:Page Values           float64
SeasonalPurchase                   float64
Month_SeasonalPurchase              object
OS                                 float64
SearchEngine                       float64
Zone                               float64
Type of Traffic                    float64
CustomerType                        object
Gender                              object
Cookies Setting                     object
Education                           object
Marital Status                      object
WeekendPurchase                    float64
dtype: object

In [5]:
x_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14731 entries, 0 to 14730
Data columns (total 21 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   HomePage                         14578 non-null  float64
 1   HomePage_Duration                14581 non-null  float64
 2   LandingPage                      14578 non-null  float64
 3   LandingPage_Duration             14596 non-null  float64
 4   ProductDescriptionPage           14608 non-null  float64
 5   ProductDescriptionPage_Duration  14564 non-null  float64
 6   GoogleMetric:Bounce Rates        14580 non-null  float64
 7   GoogleMetric:Exit Rates          14602 non-null  float64
 8   GoogleMetric:Page Values         14599 non-null  float64
 9   SeasonalPurchase                 14581 non-null  float64
 10  Month_SeasonalPurchase           14587 non-null  object 
 11  OS                               14597 non-null  float64
 12  SearchEngine      

In [6]:
result=x_train.dtypes
print(result)
x_train.describe().T


HomePage                           float64
HomePage_Duration                  float64
LandingPage                        float64
LandingPage_Duration               float64
ProductDescriptionPage             float64
ProductDescriptionPage_Duration    float64
GoogleMetric:Bounce Rates          float64
GoogleMetric:Exit Rates            float64
GoogleMetric:Page Values           float64
SeasonalPurchase                   float64
Month_SeasonalPurchase              object
OS                                 float64
SearchEngine                       float64
Zone                               float64
Type of Traffic                    float64
CustomerType                        object
Gender                              object
Cookies Setting                     object
Education                           object
Marital Status                      object
WeekendPurchase                    float64
dtype: object


,count,mean,std,min,25%,50%,75%,max
HomePage,14578.0,2.250240,3.288042,0.0,0.000000,1.000000,3.000000,27.000000
HomePage_Duration,14581.0,79.300762,179.374699,0.0,0.000000,5.000000,91.000000,3398.750000
LandingPage,14578.0,0.490739,1.252376,0.0,0.000000,0.000000,0.000000,24.000000
LandingPage_Duration,14596.0,33.455943,140.146256,0.0,0.000000,0.000000,0.000000,2549.375000
ProductDescriptionPage,14608.0,31.559488,44.897089,0.0,7.000000,17.500000,38.000000,705.000000
ProductDescriptionPage_Duration,14564.0,1184.346084,2009.496307,0.0,173.187500,584.333333,1434.255128,63973.522230
GoogleMetric:Bounce Rates,14580.0,0.023366,0.050011,0.0,0.000000,0.003478,0.018182,0.200000
GoogleMetric:Exit Rates,14602.0,0.044664,0.049912,0.0,0.014502,0.026406,0.050000,0.200000
GoogleMetric:Page Values,14599.0,4.812620,16.887366,0.0,0.000000,0.000000,0.000000,361.763742
SeasonalPurchase,14581.0,0.064083,0.202583,0.0,0.000000,0.000000,0.000000,1.000000


In [7]:
num_feature=['HomePage',
 'HomePage_Duration',
 'LandingPage',
 'LandingPage_Duration',
 'ProductDescriptionPage',
 'ProductDescriptionPage_Duration',
 'GoogleMetric:Bounce Rates',
 'GoogleMetric:Exit Rates',
 'GoogleMetric:Page Values',
 'SeasonalPurchase',
 'OS',
 'SearchEngine',
 'Zone',
 'Type of Traffic',
 'WeekendPurchase']
cat_features=['Month_SeasonalPurchase',
 'CustomerType',
 'Gender',
 'Cookies Setting',
 'Education',
 'Marital Status']

len(num_feature)


15

# **FILLING MISSING VALUES IN CATEGORIAL VARIABLES**

In [8]:
for i in cat_features:
    #print(type(x_train))
    print(i,":",x_train[i].unique())
    print(i,":",x_test[i].unique())

Month_SeasonalPurchase : ['Feb' 'Mar' 'May' 'Oct' 'June' 'Jul' 'Aug' 'Nov' 'Sep' 'Dec' nan]
Month_SeasonalPurchase : ['Feb' 'Mar' 'May' 'June' 'Oct' 'Nov' 'Sep' 'Aug' 'Jul' 'Dec' nan]
CustomerType : ['Returning_Visitor' 'New_Visitor' 'Other' nan]
CustomerType : ['Returning_Visitor' 'New_Visitor' 'Other' nan]
Gender : ['Not Specified' 'Female' 'Male' nan]
Gender : ['Not Specified' 'Female' 'Male' nan]
Cookies Setting : ['Deny' 'ALL' 'Required' nan]
Cookies Setting : ['Deny' 'ALL' 'Required' nan]
Education : ['Not Specified' 'Others' 'Diploma' 'Graduate' nan]
Education : ['Not Specified' 'Graduate' 'Diploma' 'Others' nan]
Marital Status : ['Other' 'Married' 'Single' nan]
Marital Status : ['Single' 'Married' 'Other' nan]


In [9]:
for i in cat_features:
    a=str(x_train[i].mode())
    b=str(x_test[i].mode())
    x_train[i]=x_train[i].fillna(a)
    x_test[i]=x_test[i].fillna(b)
print(x_train.isnull().sum())
print(x_test.isnull().sum())

HomePage                           153
HomePage_Duration                  150
LandingPage                        153
LandingPage_Duration               135
ProductDescriptionPage             123
ProductDescriptionPage_Duration    167
GoogleMetric:Bounce Rates          151
GoogleMetric:Exit Rates            129
GoogleMetric:Page Values           132
SeasonalPurchase                   150
Month_SeasonalPurchase               0
OS                                 134
SearchEngine                       122
Zone                               117
Type of Traffic                    143
CustomerType                         0
Gender                               0
Cookies Setting                      0
Education                            0
Marital Status                       0
WeekendPurchase                    121
dtype: int64
HomePage                           51
HomePage_Duration                  55
LandingPage                        56
LandingPage_Duration               75
ProductDescripti

# **CONVERTNG CATEGORICAL TO NUMERICAL VARIABLES**

In [10]:
for i in cat_features:
    a=pd.get_dummies(x_train[i])
    x_train=x_train.drop([i],axis=1)
    x_train=pd.concat([x_train,a],axis=1,join="inner").reindex(x_train.index)


In [11]:
for i in cat_features:
    a=pd.get_dummies(x_test[i])
    x_test=x_test.drop([i],axis=1)
    x_test=pd.concat([x_test,a],axis=1,join="inner").reindex(x_test.index)

# **HANDLING MISSING VALUES AND SCALING THE DATA**

In [12]:
si=SimpleImputer(strategy="mean")
x_train=si.fit_transform(x_train)
x_test=si.fit_transform(x_test)

ss=StandardScaler()
x_train=ss.fit_transform(x_train)
x_test=ss.fit_transform(x_test)

#Checking for NULL Values
df1=pd.DataFrame(x_train)
df2=pd.DataFrame(x_test)
df1.isnull().sum(),df2.isnull().sum()

(0     0
 1     0
 2     0
 3     0
 4     0
 5     0
 6     0
 7     0
 8     0
 9     0
 10    0
 11    0
 12    0
 13    0
 14    0
 15    0
 16    0
 17    0
 18    0
 19    0
 20    0
 21    0
 22    0
 23    0
 24    0
 25    0
 26    0
 27    0
 28    0
 29    0
 30    0
 31    0
 32    0
 33    0
 34    0
 35    0
 36    0
 37    0
 38    0
 39    0
 40    0
 41    0
 42    0
 43    0
 44    0
 45    0
 46    0
 dtype: int64,
 0     0
 1     0
 2     0
 3     0
 4     0
 5     0
 6     0
 7     0
 8     0
 9     0
 10    0
 11    0
 12    0
 13    0
 14    0
 15    0
 16    0
 17    0
 18    0
 19    0
 20    0
 21    0
 22    0
 23    0
 24    0
 25    0
 26    0
 27    0
 28    0
 29    0
 30    0
 31    0
 32    0
 33    0
 34    0
 35    0
 36    0
 37    0
 38    0
 39    0
 40    0
 41    0
 42    0
 43    0
 44    0
 45    0
 46    0
 dtype: int64)

In [13]:
y_train=y_train.astype(int)

In [14]:
y_train.head()
y_train.isna().sum()

0

# **LOGISTIC REGRESSION**

In [15]:
from sklearn.linear_model import LogisticRegression 

model1 = LogisticRegression(random_state=42)

classifier1 = model1.fit(x_train,y_train)

score1 = model1.score(x_train,y_train)

train_pred1 = model1.predict(x_train)

test_pred1 = model1.predict(x_test)

print(score1)

0.6586110922544295


# **RANDOMFORESTCLASSIFIER** 

In [16]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,f1_score,r2_score
model2 = RandomForestClassifier() 


classifier2 = model2.fit(x_train,y_train)

score2 = model2.score(x_train,y_train)

train_pred2 = model2.predict(x_train)

y_test_pred2 = model2.predict(x_test)

print(score2)

f1_score(y_train,train_pred2)


0.8107392573484489


0.7259681541183408

# **KNeighborsClassifer**

In [17]:
from sklearn.neighbors import KNeighborsClassifier

model3=KNeighborsClassifier()

classifier3 = model3.fit(x_train,y_train)

score3 = model3.score(x_train,y_train)

train_pred3 = model3.predict(x_train)

y_test_pred3 = model3.predict(x_test)

print(score3)

f1_score(y_train,train_pred3)

0.6945896408933542


0.5043516580367964

# **Support Vector Classifier**

In [18]:
from sklearn.svm import SVC
model4 = SVC()
classifier4 = model4.fit(x_train,y_train)

score4 = model4.score(x_train,y_train)

train_pred4 = model4.predict(x_train)

y_test_pred4 = model4.predict(x_test)

print(score4)

f1_score(y_train,train_pred4)

0.6761251781956419


0.3425657985393413

In [19]:
test_pred5 = model4.predict(x_test)
train_pred5=model4.predict(x_train)
print(accuracy_score(y_train,train_pred5))

0.6761251781956419


In [20]:
submission=pd.DataFrame(columns=['id','Made_Purchase'])
submission['id']=[i for i in range(len(test_pred5))]
submission['Made_Purchase']=test_pred5.astype(bool)
submission.to_csv('submission.csv',index=False)


print(submission[submission==True].count(),submission[submission==False].count())


id                 1
Made_Purchase    681
dtype: int64 id                  1
Made_Purchase    5918
dtype: int64
